In [81]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
Incidents_upto=pd.read_excel("Incidents up to 30.11.2021.xlsx")
new_header= Incidents_upto.iloc[0]
Incidents_upto=Incidents_upto[1:]
Incidents_upto.columns=new_header
Incidents_upto.head()

,No.,Date,Time,Type,Lat,Long,Vessel,Vessel Status,Boarding,Damage to property,Damage to human life,Weapon,Port/ Out Side Port Limits,Country,Location,Summary
1,1,2021-01-03 00:00:00,2021-07-19 09:15:00,Military,25.12,64.30,NaN,NaN,NaN,NaN,NaN,NaN,Open Sea,Pakistan,Indian Ocean,A Live Firing Practice
2,2,2021-01-03 00:00:00,2021-07-19 19:53:00,Robbery,18.38,-72.37,Bulk,Anchored,Yes,No,No,Yes,Port,Haiti,Americas,The security patrol discovered up to six robbe...
3,3,2021-01-04 00:00:00,2021-07-19 01:45:00,Robbery,-4.4253,11.4406,Container,Anchored,Yes,Yes,No,No,Port,Congo,West Africa,Three robbers onboard. Two ropes had been stolen.
4,4,2021-01-04 00:00:00,2021-07-19 02:40:00,Robbery,-0.1508,117.3403,Bulk,Anchored,Yes,No,No,No,Port,Indonesia,South East Asia,Three robbers armed with knives managed to boa...
5,5,2021-01-04 00:00:00,2021-07-19 06:15:00,General Alert,26.36,56.09,Tanker,Underway,No,No,No,No,Open Sea,Oman,Indian Ocean,An interaction has occurred between a Korean f...


In [83]:
wld_trs_ports=pd.read_excel("wld_trs_ports_wfp.xls")
wld_trs_ports.head()

,FID,portname,code,prttype,prtsize,status,maxdepth,maxlength,annualcapacitymt,humuse,locprecision,latitude,longitude,iso3,iso3_op,country,lastcheckdate,remarks,url_lca,source,createdate,updatedate,geonameid,gdb_geomattr_data,shape
0,wld_trs_ports_wfp.14314,Watsi-Genge,NaN,River,Very Small,Unknown,NaN,NaN,NaN,Unknown,unknown,-0.94560,20.62966,COD,COD,Democratic Republic of the Congo,NaT,NaN,NaN,NaN,2021-02-24 11:52:47.493,2021-02-24 11:52:47.493,204280,NaN,POINT (20.629661913000007 -0.9456024360000015)
1,wld_trs_ports_wfp.14315,Charlotte (Skidegate),CASKI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,accurate,53.24742,-132.00969,CAN,CAN,Canada,NaT,NaN,NaN,NaN,2021-02-24 11:52:47.493,2021-02-24 11:52:47.493,6148858,NaN,POINT (-132.009692535 53.247424029)
2,wld_trs_ports_wfp.14316,Homer,USHOI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,accurate,59.60159,-151.41553,NaN,USA,NaN,NaT,NaN,NaN,NaN,2021-02-24 11:52:47.493,2021-02-24 11:52:47.493,5864145,NaN,POINT (-151.415532437 59.60158521400001)
3,wld_trs_ports_wfp.14317,Hartsdale,USHAS,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,accurate,40.70610,-74.00210,NaN,USA,NaN,NaT,NaN,NaN,NaN,2021-02-24 11:52:47.493,2021-02-24 11:52:47.493,5120141,NaN,POINT (-74.00209774799998 40.70610420999998)
4,wld_trs_ports_wfp.14318,Killingholme,GBKGH,Sea,Unknown,Unknown,NaN,NaN,NaN,Unknown,accurate,53.64561,-0.21512,NaN,GBR,NaN,NaT,NaN,NaN,NaN,2021-02-24 11:52:47.493,2021-02-24 11:52:47.493,2641323,NaN,POINT (-0.2151185050000208 53.64560881900002)


In [84]:
#converting in  radians
wld_trs_ports['latitude'] = np.radians(wld_trs_ports['latitude'])
wld_trs_ports['longitude'] = np.radians(wld_trs_ports['longitude'])
wld_trs_ports.filter(['latitude','longitude'])


,latitude,longitude
0,-0.016504,0.360055
1,0.929343,-2.304004
2,1.040244,-2.642700
3,0.710455,-1.291580
4,0.936293,-0.003755
...,...,...
3570,-0.037649,0.506458
3571,-0.079057,0.324699
3572,-0.055484,0.282726
3573,-0.101753,0.303687


In [85]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('haversine')
wld_trs_ports[['latitude','longitude']].to_numpy()
distance=dist.pairwise(wld_trs_ports [['latitude','longitude']].to_numpy())*6373
print(distance)

[[    0.         13680.76886184 13460.80858451 ...   551.5794532
    650.91096231  1510.89176488]
 [13680.76886184     0.          1379.74267239 ... 13739.20212796
  14062.53140728 12931.1725877 ]
 [13460.80858451  1379.74267239     0.         ... 13655.9660283
  13965.15615162 12424.58414331]
 ...
 [  551.5794532  13739.20212796 13655.9660283  ...     0.
    323.54246935  2041.33814675]
 [  650.91096231 14062.53140728 13965.15615162 ...   323.54246935
      0.          2152.86587495]
 [ 1510.89176488 12931.1725877  12424.58414331 ...  2041.33814675
   2152.86587495     0.        ]]


In [86]:
dataframe=pd.DataFrame(columns=wld_trs_ports.portname, index=wld_trs_ports.portname)
dataframe.shape

(3575, 3575)

In [87]:
DataFrame_distance=pd.DataFrame(distance,columns=wld_trs_ports.portname, index=wld_trs_ports.portname)
DataFrame_distance

portname,Watsi-Genge,Charlotte (Skidegate),Homer,Hartsdale,Killingholme,Shoreham,Southend,Santa Fe,Punta De Palmas,Moin Bay,Canaport,Winslow,West Foreland,Redwood City,Shawano,Mars,Sullom Voe,Salina,Eleusis,Syros (Syra),Cebu,Coco Grove Nature Resort,Pointe Des Galets,Sulina (Port Et Zone Franche),Damman,Tijuana,Jose Ignacio Terminal,Khark Island,Bandar Mahshahr,Porto Foxi (Sarroch),Rharous,Ambohitsara,Pomala,Telok Ayer,Lifake,Escravos,My Tho,Yafala,Gutthom,Vum,...,Gerge Town,Mocmboa da Praia,Kipindi,Ipole,Nkuvu,Ulindi,NaN,Lilanga,Makayabo,Itoko,Karhanywa,NaN,NaN,Boza,Mabunde,De Bwina,Annicet / Ntantu,Kunde,Shwishwi,Mushimbake,Lumbu,Batempa,Olympia,NaN,Lusanga,Kimpasa,NaN,Fabieza,Bwajingi,Boburu,Nkora,Kamtsha,Kagobe,Kirabo,Nsulo,Kihara,Mazinga,Malela,NaN,Nyaroump
portname,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Watsi-Genge,0.000000,13680.768862,13460.808585,10469.743267,6370.647304,6089.679917,6131.056004,9134.749767,10277.420742,11530.204744,9820.028167,13732.891082,13341.285176,14453.855463,11994.751270,12223.190881,7085.898001,4433.488117,4346.738684,4292.363186,11536.388937,11522.661906,4371.208372,5205.029784,4399.701926,14379.272730,8608.990268,4606.341050,4619.092879,4608.434430,3170.143913,3746.960263,11219.077054,9891.736184,279.754401,1861.970999,9559.056123,172.149568,1447.505858,371.946046,...,10705.188995,2466.404190,463.703357,373.912518,951.860088,770.688425,635.852749,313.681037,409.597001,119.180872,951.023902,607.238199,650.961306,950.663334,287.793249,948.096381,457.054456,630.637561,951.740505,1004.414021,437.025620,575.587924,378.217805,650.910962,478.909443,615.576049,686.402825,539.641712,948.212658,391.634522,943.413235,421.690354,942.641711,944.962851,948.122246,942.347368,457.781395,551.579453,650.910962,1510.891765
Charlotte (Skidegate),13680.768862,0.000000,1379.742672,4480.655104,7326.138107,7598.076284,7562.255121,11668.619786,7134.913110,6478.478396,4702.518488,917.326961,1445.520908,1906.291462,4355.570836,4181.923986,6639.868029,9324.009162,9596.940882,9692.618903,9948.969967,9957.937720,16372.875209,8937.575093,11149.968632,2594.570877,12292.739386,10846.156562,10708.880934,9088.672872,10880.222302,16419.355285,11469.336753,11922.362191,13521.951657,12348.739013,11067.776484,13778.870546,13194.829465,13916.767414,...,5712.213568,15310.347890,13995.036509,13917.637752,14084.511409,14086.348907,14044.668447,13449.322767,14077.735674,13726.811524,14080.170041,14010.183518,14063.368023,14087.748664,13437.751161,14089.616219,13977.607381,13927.753357,14079.324038,14289.236797,13797.199066,14225.143795,13927.039673,14062.531407,14146.183821,13908.733991,14106.817584,13857.550814,14056.087880,13333.849807,14056.046578,13996.419733,14057.341509,14066.473016,14055.695807,14076.832295,13977.485941,13739.202128,14062.531407,12931.172588
Homer,13460.808585,1379.742672,0.000000,5533.784198,7165.221012,7465.743453,7408.342331,13042.572782,8435.902120,7841.903151,5580.476472,2297.030530,124.738891,3213.284524,5681.607048,5521.689733,6420.181027,9030.283076,9150.365900,9222.492400,8665.195943,8673.297503,15228.255620,8368.507771,10242.555776,3926.952091,13660.229364,9941.875469,9825.186697,8897.321675,11077.924633,15433.428637,10242.765205,10590.354527,13250.097188,12480.391506,9696.980663,13520.435015,12666.597853,13762.050510,...,6971.190101,14564.294699,13851.653472,13763.361120,13637.402045,13689.421206,13946.296556,13184.394676,13870.389651,13477.963090,13632.980713,13909.850968,13965.828657,13641.188735,13187.319966,13643.877891,13836.426288,13847.718172,13631.885942,13848.502632,13679.997464,13939.438359,13771.904655,13965.156152,13939.613992,13827.028400,14011.050152,13761.324689,13608.071336,13169.432464,13609.288143,13837.289720,13610.866712,13619.970447,13607.678945,13631.720657,13836.579641,13655.966028,13965.156152,12424.584143
Hartsdale,10469.743267,4480.655104,5533.784198,0.000000,5495.296917,5586.703701,5626.559874,8163.681743,3374.430186,3529.845353,822

In [88]:

list(DataFrame_distance.columns)
list_DataFrame_ports=list(DataFrame_distance.columns)

In [89]:
dict_df_distance=DataFrame_distance.to_dict()

In [90]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


def near_by_incident(incident_point, test_point):
  #center_point = [{'lat': -7.7940023, 'lng': 110.3656535}]
  #test_point = [{'lat': -7.79457, 'lng': 110.36563}]

  lat1 = center_point[0]['lat']
  lon1 = center_point[0]['lng']
  lat2 = test_point[0]['lat']
  lon2 = test_point[0]['lng']

  radius = 12000 # in kilometer
  a = haversine(lon1, lat1, lon2, lat2)
  if a <= radius:
      return True
  else: return False

def near_any_incident(port,incidents,ports):
  all_incidents = [{'lat':incidents['Lat'][i+1], 'lng':incidents['Long'][i+1]} for i in range(len(incidents))]
  try: point = [{'lat':ports[ports['portname'] == port]['latitude'].iloc[0], 'lng':ports[ports['portname'] == port]['longitude'].iloc[0]}] 
  except: return True
  for incident in all_incidents:
    if near_by_incident(incident,point): return True
  return False

In [91]:
import sys 
from heapq import heapify, heappush, heappop

def dijsktra(graph,src,dest):
    inf = float(sys.maxsize)
    node_data={}
    for element in list_DataFrame_ports:
        node_data[element]={'cost':inf,'pred':[]}
      
    node_data[src]['cost']=0
    visited=[]
    temp=src
    for i in range(3575):
        if temp not in visited:
            visited.append(temp)
            min_heap=[]
            for j in graph[temp]:
                if j == 'Soma': continue
                if j not in visited:
                    cost=node_data[temp]['cost']+graph[temp][j]
                if cost<node_data[j]['cost'] and not near_any_incident(j,Incidents_upto,wld_trs_ports):
                    node_data[j]['cost']=cost
                    node_data[j]['pred'].append(temp)
                heappush(min_heap,(node_data[j]['cost'],j))
        heapify(min_heap)
        temp=min_heap[0][1]
    node_data[dest]['pred'].append(dest)
    print("Shortest Distance: " + str(node_data[dest]['cost']))
    print("Shortest Path: " + str(node_data[dest]['pred']))
                        
if __name__ =="__main__":
    graph=dict_df_distance
    
    dijsktra(graph,'Hartsdale','Dili')

Shortest Distance: 15955.576484492898
Shortest Path: ['Hartsdale', 'Dili']
